In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import os
import time
from skimage.feature import local_binary_pattern
from skimage.exposure import histogram
import skimage.io as io
from sklearn import svm

In [9]:
def cutLines(gray):
    edges = cv2.Canny(gray,90,100,apertureSize = 3)
    minLineLength=55
    lines = cv2.HoughLinesP(image=edges,rho=1,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=minLineLength,maxLineGap=0)
 
    linesNew=[ line[0][1] for line in lines if line[0][1]==line[0][3]]
    linesNew=sorted(linesNew)
    finalList=[linesNew[0]]
    for i in range(1,len(linesNew)):
        if linesNew[i]-linesNew[i-1]>=100:
            finalList.append(linesNew[i])
    # io.imshow(gray[finalList[1]:finalList[2],:])
    return gray[finalList[1]:finalList[2],:]

In [10]:
# def lines_segments(image):
#     gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
#     kernel = np.ones((10,200), np.uint8)
#     img_dilation = cv2.dilate(thresh, kernel, iterations=1)
#     _, ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
#     images = []
#     flag=True
#     y1 = 10e5
#     y2 = -10e5 
#     height = 0 
#     for i, ctr in enumerate(sorted_ctrs):
#         x, y, w, h = cv2.boundingRect(ctr) 
#         roi = image[y:y+h, x:x+w]
#         if(len(sum(roi))>200 and h>40): #20 is thershold you can change this, this is for small lines and points, remove them from return images 
#             images.append(roi)
#             y2=max(y2,y+h)
#             y1=min(y1,y) 
            
#             #io.imsave(str(i)+'.png',roi)
#     #--UnComment-- in case you want to return the whole  image not just lines 
#     #if(y2>0):
#     #    croped = gray[y1:y2, ::]
#     #else:
#     #   croped = gray
#     return images

def lines_segments(image):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((5,100), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    _, ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    images = []
    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr) 
        roi = gray[y:y+h, x:x+w]
        if(len(roi)>25 and y+h<x+w): #20 is thershold you can change this, this is for small lines and points, remove them from return images 
            images.append(roi)
            # io.imsave(str(i)+'.png',roi)
    # write
    return images

In [11]:
def LBP (images, stride = 1):
    hist = np.zeros(256)
    for img in images:
        lbp = local_binary_pattern(img, 8, 1)   #‘default’, ‘ror’, ‘uniform’, ‘var’, nri_uniform
        hist += np.asarray(histogram(lbp)[1]).astype(int)
    return hist    

In [13]:
def read_test_case(path):
    images = []
    labels = []
    test_image = ''
    for root, dirs, files in os.walk(path):
        for d in dirs:
            for root2, dir2s, file2s in os.walk(path + '/' + d):  
                for filename in file2s:
                    print(path +'/'+ d + '/'+ filename)
                    image = cv2.imread(path + '/' + d  + '/' + filename)
                    images.append(image)
                    labels.append(d) 
                break    
        test_image = cv2.imread(path + '/' + 'test.png')           
        break
    return images, labels, test_image

result_file = open("results.txt", 'a')
time_file = open("time.txt", 'a')
def read_ALL(path):
    #path = 'data'
    for root, dirs, files in os.walk(path):
        for d in dirs:
            images, labels, test_image = read_test_case(path+'/'+d)
            features = []
            start = time.time()
            for i in range(len(images)):
                img = cutLines(images[i])
                image_lines = lines_segments(img)
                feature = LBP(image_lines)
                features.append(feature)

            classifier = svm.SVC(C=5.0, gamma='auto', probability=True)
            classifier.fit(features, labels)

            img = cutLines(test_image)
            image_lines = lines_segments(img)
            feature = LBP(image_lines)
            # print(feature.shape)
            y_pred = classifier.predict(feature.reshape(1,256))
            # print(y_pred[0])
            end = time.time()
            result_file.write(str(y_pred[0]) +'\n')

            t = (end - start)
            time_file.write(str(round(t,2)))
            time_file.write('\n')
        break

    result_file.close()
    time_file.close()

read_ALL('data')

data/01/1/1.png
data/01/1/2.png
data/01/2/1.png
data/01/2/2.png
data/01/3/1.png
data/01/3/2.png
data/02/1/1.png
data/02/1/2.png
data/02/2/1.png
data/02/2/2.png
data/02/3/1.png
data/02/3/2.png
data/03/1/1.png
data/03/1/2.png
data/03/2/1.png
data/03/2/2.png
data/03/3/1.png
data/03/3/2.png
data/04/1/1.png
data/04/1/2.png
data/04/2/1.png
data/04/2/2.png
data/04/3/1.png
data/04/3/2.png
data/05/1/1.png
data/05/1/2.png
data/05/2/1.png
data/05/2/2.png
data/05/3/1.png
data/05/3/2.png
data/06/1/1.png
data/06/1/2.png
data/06/2/1.png
data/06/2/2.png
data/06/3/1.png
data/06/3/2.png
data/07/1/1.png
data/07/1/2.png
data/07/2/1.png
data/07/2/2.png
data/07/3/1.png
data/07/3/2.png
data/08/1/1.png
data/08/1/2.png
data/08/2/1.png
data/08/2/2.png
data/08/3/1.png
data/08/3/2.png
data/09/1/1.png
data/09/1/2.png
data/09/2/1.png
data/09/2/2.png
data/09/3/1.png
data/09/3/2.png
data/10/1/1.png
data/10/1/2.png
data/10/2/1.png
data/10/2/2.png
data/10/3/1.png
data/10/3/2.png
